Примечание: для данного датасета запись в таблице будет определяться как "арендное жилье"

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/seattle.csv')

In [3]:
df

#### 1. Сгруппируйте арендное жилье по параметру room_type и выведите количество в каждом типе.

In [4]:
df.groupby(by=df.room_type).room_id.count()

#### 2. Посчитайте среднее количество спален (bedrooms) и ванн (bathrooms)  (каждого по отдельности), где аренда (price) < 90 у.е.

In [5]:
df[df.price < 90][['bedrooms', 'bathrooms']].mean()

#### 3. Выведите топ-5 апартаментов (room_type = "Entire home/apt") с наибольшим количеством отзывов (reviews)

In [6]:
df[df.room_type == "Entire home/apt"].sort_values(by='reviews', ascending=False).head()

#### 4. Найдите хозяина (host_id) с максимальным числом сдаваемого в аренду жилья

In [7]:
df.groupby(by=df.host_id).host_id.count().sort_values(ascending=False).head(1) 

#### 5. Найдите хозяина, у которого наибольшее число отзывов, рейтинг (overall_satisfaction) которых > 4

In [8]:
df[df.overall_satisfaction > 4].groupby(by=df.host_id).count().sort_values(by='reviews', ascending=False).host_id.head(1)

#### 6. Найдите минимальную и максимальную стоимость видов жилья, где число проживающих (accommodates) варьирует от 2 до 6 человек включительно

In [9]:
pd.pivot_table(df[(df.accommodates >= 2) & (df.accommodates <= 6)],
               index=df.room_type, 
               values='price', 
               aggfunc=[max, min])

,max,min
,price,price
room_type,,
Entire home/apt,960,21
Private room,999,16
Shared room,180,15


#### 7. Посчитайте количество комнат (ванна + спальня) для каждой записи (нужно добавить новую колонку total_rooms). Найдите максимальное значение total_rooms для каждой из категорий (room_type) арендного жилья.

In [10]:
df['total_rooms'] = df.bedrooms + df.bathrooms

pd.pivot_table(df,
               index=df.room_type,
               values='total_rooms',
               aggfunc=max)

,total_rooms
room_type,
Entire home/apt,14.0
Private room,8.0
Shared room,9.0


#### 8. Посчитайте процентное соотношение числа отзывов по следующим категориям: 
* overall_satisfaction >= 0 and overall_satisfaction < 1
* overall_satisfaction >= 1 and overall_satisfaction < 2 
* overall_satisfaction >= 2 and overall_satisfaction < 3
* overall_satisfaction >= 3 and overall_satisfaction < 4
* overall_satisfaction >= 4 and overall_satisfaction < 5
* overall_satisfaction == 5

In [11]:
answ = pd.concat([df.overall_satisfaction.fillna(6).astype(int), df.reviews], axis=1).groupby('overall_satisfaction')\
                                                                                     .sum()\
                                                                                     .reset_index()


answ.reviews = answ.reviews / answ.reviews.sum()

In [12]:
answ

,overall_satisfaction,reviews
0,2,0.000008
1,3,0.000820
2,4,0.231083
3,5,0.765450
4,6,0.002639


In [13]:
def print_answ(x):
    try:
        return float(answ[answ.overall_satisfaction == x].reviews)
    except TypeError:
        return 0.
    
for i in range(7):
    if i == 6:
        print(0, ': ', round(print_answ(i), 3))
        continue
    print(i+1, ': ', round(print_answ(i), 3))

1 :  0.0
2 :  0.0
3 :  0.0
4 :  0.001
5 :  0.231
6 :  0.765
0 :  0.003


#### 9. В какой час было произведено наибольшее число изменений записи (last_modified)? 


In [14]:
df.groupby(pd.to_datetime(df.last_modified).dt.hour)\
  .count()\
  .sort_values(by='Unnamed: 0', ascending=False)\
  .last_modified\
  .head(1)

last_modified
3    5409
Name: last_modified, dtype: int64

#### 10. Найдите топ-10 хозяев, которые заработали больше всех денег и топ-10, которые заработали меньше всего.

In [15]:
stat = df.groupby(by=df.host_id)\
         .sum()\
         .sort_values(by='price', ascending=False)\
         .price.to_frame()\
         .reset_index()

stat.head(10)

,host_id,price
0,8534462,32891
1,50550045,14730
2,82961680,11259
3,138924514,9938
4,4962900,7547
5,207109604,7312
6,114353388,7191
7,74305,5957
8,734114,5900
9,140236800,5344


In [16]:
stat.tail(10)

,host_id,price
4372,180907024,25
4373,137392202,25
4374,87606372,25
4375,208552119,25
4376,57027116,24
4377,189279829,24
4378,206723309,23
4379,196044187,22
4380,2390150,19
4381,154167958,17
